In [1]:
import piano_reduction.tools as pr
import piano_reduction.compute_features as cf
from piano_reduction.classes import ScoreData, Reduction
from sklearn import metrics
import numpy as np

Using TensorFlow backend.


In [2]:
reductionsn = {}

In [3]:
for i in [7, 8]:
    for j in [0, 1, 2, 3, 4]:
        try:
            reductionsn['%d GRU_1 -old %d' % (i, j)] = 'models/20190415c/GRU_1 -old %d %d.pkl' % (i, j)
        except:
            pass
        try:
            reductionsn['%d GRU_2 -old %d' % (i, j)] = 'models/20190415c/GRU_2 -old %d %d.pkl' % (i, j)
        except:
            pass
        try:
            reductionsn['%d GRU_1 -new %d' % (i, j)] = 'models/20190415c/GRU_1 -new %d %d.pkl' % (i, j)
        except:
            pass
        try:
            reductionsn['%d GRU_2 -new %d' % (i, j)] = 'models/20190415c/GRU_2 -new %d %d.pkl' % (i, j)
        except:
            pass

In [32]:
for i in [7]:
    reductions['%d GRU_1 -old' % i] = Reduction.load('models/20190410w/GRU_1 -chord %d.pkl' % i)
    reductions['%d GRU_2 -old' % i] = Reduction.load('models/20190410w/GRU_2 -chord %d.pkl' % i)
    reductions['%d GRU_1 -new' % i] = Reduction.load('models/20190410w/GRU_1 -new %d.pkl' % i)
    reductions['%d GRU_2 -new' % i] = Reduction.load('models/20190410w/GRU_2 -new %d.pkl' % i)

In [43]:
for i in reductions:
    if 'GRU_1' in i:
        reductions[i].params['length'] = 20

In [6]:
for i in [7]:
    dfs = pr.load_merged_df(i, use_cache=False)
    score = ScoreData(dfs)
    score.save('score_data/%d.pkl' % i)

Creating df/input_with_chords/i_0007_Tchaikovsky_nutcracker_marchwithchord.musicxml
P1: Violin I: Violin 0
P2: Flute: Flute 0
P3: Horn in F: Horn in F -7
P4: Violoncello: Violoncello 0
P5: A Clarinet: A Clarinet -3
P6: B♭ Clarinet: B♭ Clarinet -2
Creating df/output/o_0007_Tchaikovsky_nutcracker_march.xml
P1: Piano: Piano 0
P1: Piano: Piano 0
Creating df/merged/7.pkl


In [4]:
def check_data(name, reds, pos, threshold=0.1):
    _, new_data = ScoreData.load(name).split_in(pos, pos + 0.2)
    
    features = ['active_rhythm', 'bass_line', 'entrance_effect', 'highest', 'in_chord', 'lowest', 'occurrence', 
       'onset_after_rest', 'rhythm_variety', 'strong_beats', 'sustained_rhythm', 'vertical_doubling', 'duration_length', 'offset']
    from sklearn.linear_model import LogisticRegression
    from sklearn.naive_bayes import MultinomialNB
    
    for i in reds:
        if 'GRU' in i:
            new_data = reductions[i].predict(new_data, threshold=threshold, name='y_pred_%s' % i)
        else:
            new_data = reductions[i].predict_2(new_data, threshold=threshold, name='y_pred_%s' % i)
    return new_data

In [ ]:
print('%15s| %25s\t%20s\t%20s\t%20s' % ('Testing Data','Model Used', 'F1 score', 'Accuracy', 'Jaccard Similarity'))
names = []
for i in [7]:
    names.append('score_data/%d.pkl' % i)
for i in []:
    names.append('score_data/cosi_%d.pkl' % i)
reductions = {}
for k in ['8', '7']:
    for gru in ['GRU_1 -new', 'GRU_1 -old', 'GRU_2 -new', 'GRU_2 -old']:
    
        from keras import backend as K
        K.clear_session()
        if k == '7':
            name = 'score_data/7.pkl'
        else:
            name = 'score_data/cosi_%s.pkl' % k
        reductions = {}
        for i in reductionsn:
            if gru in i and i[0] == k:
                reductions[i] = Reduction.load(reductionsn[i])
        avg = np.array([0.0, 0.0, 0.0])
        for j in [0, 1, 2, 3, 4]:
            i = '%s %s %d' % (k, gru, j)
            new_data = check_data(name, [i], j * 0.2, threshold=0.7)
            b1 = new_data.to_binary(col='y_train')
            b2 = new_data.to_binary(col='y_pred_%s' % i)
            scores = [metrics.f1_score(new_data.df['y_train'], new_data.df['y_pred_%s' % i]),
                      metrics.accuracy_score(new_data.df['y_train'], new_data.df['y_pred_%s' % i]),
                      pr.jaccard_similarity(b1, b2)
                     ]
            if 'GRU' in i:
                i = '%s using %s.pkl' % (i[2:], k)
            #elif 'Attempt 2 both ways' in i:
            #    i = '2 GRU using cosi_%c' % i[0]
            #print('%15s: %25s\t%20.9f\t%20.9f\t%20.9f' % tuple([name[11:], i] + scores))
            avg += np.array(scores)
        #print('%15s  %25s\t%20.9f\t%20.9f\t%20.9f' % tuple(['', 'Average'] + list(avg / 5.0)))
        print('%.9f\t%.9f\t%.9f' % tuple(list(avg / 5.0)))
        #print()
        

In [ ]:
1

In [44]:
print('%25s\t%20s\t%20s\t%20s' % ('', 'F1 score', 'Accuracy', 'Jaccard Similarity'))
reds = [i for i in sorted(list(reductions), key=lambda x:x[::-1]) if 'GRU' in i]
for name in ['score_data/cosi_2.pkl', 'score_data/cosi_6.pkl', 'score_data/cosi_8.pkl']:
    new_data = check_data(name, reds)
    for i in list(reds):
        b1 = new_data.to_binary(col='y_train')
        b2 = new_data.to_binary(col='y_pred_%s' % i)
        scores = [metrics.f1_score(new_data.df['y_train'], new_data.df['y_pred_%s' % i]),
                  metrics.accuracy_score(new_data.df['y_train'], new_data.df['y_pred_%s' % i]),
                  pr.jaccard_similarity(b1, b2)
                 ]
        if 'GRU' in i:
            i = '%s using cosi_%c' % (i[2:], i[0])
        #elif 'Attempt 2 both ways' in i:
        #    i = '2 GRU using cosi_%c' % i[0]
        print('%25s\t%20.9f\t%20.9f\t%20.9f' % tuple([i] + scores))
    print()

                         	            F1 score	            Accuracy	  Jaccard Similarity
  GRU_1 -old using cosi_2	         0.803059273	         0.807476636	         0.670926518
  GRU_1 -old using cosi_6	         0.760907504	         0.743925234	         0.614084507
  GRU_1 -old using cosi_7	         0.477351916	         0.439252336	         0.313501144
  GRU_1 -old using cosi_8	         0.692015209	         0.697196262	         0.529069767
  GRU_2 -old using cosi_2	         0.859922179	         0.865420561	         0.754266212
  GRU_2 -old using cosi_6	         0.772058824	         0.768224299	         0.628742515
  GRU_2 -old using cosi_7	         0.463157895	         0.428037383	         0.301369863
  GRU_2 -old using cosi_8	         0.733870968	         0.753271028	         0.579617834
  GRU_1 -new using cosi_2	         0.759398496	         0.760747664	         0.612121212
  GRU_1 -new using cosi_6	         0.779467681	         0.783177570	         0.638629283
  GRU_1 -new using co

In [36]:
reds = [i for i in sorted(list(reductions)) if i[0] == '7' or (i[0] == '8' and 'GRU_2 -new' in i)]
#_, new_data = ScoreData.load('score_data/7.pkl').split_in(0.8, 0.9)
new_data = ScoreData.load('score_data/2.pkl')
#dfs = pr.load_df('input_with_chords', 7)
#score = ScoreData(dfs)
#score.save('score_data/fun_%d.pkl' % 0)
#new_data = score
for i in reds:
    new_data = reductions[i].predict(new_data, threshold=0.1, name='y_pred_%s' % i)


In [37]:
new_data.show_score(['x_train'] + ['y_pred_%s' % i for i in reds])

{'length': 39,
 'features': ['active_rhythm',
  'bass_line',
  'entrance_effect',
  'highest',
  'in_chord',
  'lowest',
  'occurrence',
  'onset_after_rest',
  'rhythm_variety',
  'strong_beats',
  'sustained_rhythm',
  'vertical_doubling',
  'duration_length']}